In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # import geocoder

Load DataFrame we scraped from Wikipedia

In [2]:
df = pd.read_csv('../input/Toronto Postcodes.csv', usecols=['Postcode', 'Bourough', 'Neighbourhood'])
df.head()

,Postcode,Bourough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Now we'll query ArcGIS for latitude and longtitude data.

In [13]:
# accepts a list of postcodes in Toronto, Ontario and returns a DataFrame
# with Postcode, Latitude, Longtitude as columns
# each line in the DataFrame represents one postcode
def batch_get_lat_lng(postcodes):
    df = pd.DataFrame(columns=['Postcode', 'Latitude', 'Longitude'])
    
    for code in postcodes:
        lat_lng_coords = None
        while(lat_lng_coords is None):
            g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
            lat_lng_coords = g.latlng

        df = df.append({'Postcode': code,
                        'Latitude': lat_lng_coords[0],
                        'Longitude': lat_lng_coords[1]},
                       ignore_index=True)
    
    return df

In [8]:
coords_df = batch_get_lat_lng(df.Postcode)

In [6]:
coords_df.head()

,Postcode,Latitude,Longitude
0,M3A,43.752435,-79.329268
1,M4A,43.730417,-79.313340
2,M5A,43.655120,-79.362640
3,M6A,43.723270,-79.451603
4,M7A,43.661072,-79.390895


Now let's merge the two DataFrames using the Postcode column.

In [11]:
df = pd.merge(left=df, right=coords_df, left_on='Postcode', right_on='Postcode')

In [12]:
df.head()

,Postcode,Bourough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752435,-79.329268
1,M4A,North York,Victoria Village,43.730417,-79.313340
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451603
4,M7A,Queen's Park,Queen's Park,43.661072,-79.390895


In [14]:
df.to_csv('Toronto Postcodes and coordinates')